# AI Capstone: Project 1
## *with a public non-image dataset*
Author: 0816066 官澔恩

Dataset Source: [Kaggle](https://www.kaggle.com/fedesoriano/stellar-classification-dataset-sdss17)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from imblearn.under_sampling import RandomUnderSampler

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_validate, ParameterGrid
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Data Preprocessing

In [2]:
data_path = '../input/stellar-classification-dataset-sdss17/star_classification.csv'
data = pd.read_csv(data_path)
data

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [3]:
print(data['class'].value_counts())
# labeling encode the feature 'class'
encoder = LabelEncoder()
label_encoded_classes = encoder.fit_transform(data['class'].values)
data['class'] = label_encoded_classes

GALAXY    59445
STAR      21594
QSO       18961
Name: class, dtype: int64


In [4]:
corr = data.corr()
corr_class = corr['class'].sort_values()
threshold = (corr_class > 0.02) | (corr_class < -0.02)
print(pd.DataFrame({'corr_coef': corr_class, 'meet_thres': threshold}))

to_drops = threshold.index[~threshold].tolist()
to_drops.append('class')

             corr_coef  meet_thres
r            -0.076766        True
redshift     -0.054239        True
fiber_ID     -0.041586        True
run_ID       -0.036014        True
obj_ID       -0.036012        True
field_ID     -0.034833        True
u            -0.024645        True
g            -0.020066        True
alpha        -0.011756       False
spec_obj_ID  -0.010060       False
plate        -0.010060       False
z            -0.001614       False
MJD          -0.000405       False
delta         0.014452       False
i             0.015028       False
cam_col       0.023138        True
class         1.000000        True
rerun_ID           NaN       False


`alpha`, `spec_obj_ID`, `plate`, `z`, `MJD`, `delta`, `i`, and `rerun_ID` has little relation with the target feature `class`.

Therefore, I'll drop them.

In [5]:
# drop unrelated features and the target feature
data_X = data.drop(columns=to_drops)
data_y = data['class']

In [6]:
data_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   obj_ID    100000 non-null  float64
 1   u         100000 non-null  float64
 2   g         100000 non-null  float64
 3   r         100000 non-null  float64
 4   run_ID    100000 non-null  int64  
 5   cam_col   100000 non-null  int64  
 6   field_ID  100000 non-null  int64  
 7   redshift  100000 non-null  float64
 8   fiber_ID  100000 non-null  int64  
dtypes: float64(5), int64(4)
memory usage: 6.9 MB


`run_ID`, `cam_col`, `field_ID`, and `fiber_ID` is categorical.

They have to be one-hot encoded.

In [7]:
# transform each categorical attributes into one-hot encoded ones
cat_attrs = ['run_ID', 'cam_col', 'field_ID', 'fiber_ID']
for cat_attr in cat_attrs:
    one_hot = pd.get_dummies(data_X[cat_attr]) \
                .add_prefix(f'{ cat_attr }_')
    data_X = data_X.drop(columns=cat_attr)
    data_X = data_X.join(one_hot)

data_X = data_X.values
print(f'Number of features becomes { data_X.shape[1] }.')

Number of features becomes 2297.


In [8]:
test_sizes = [0.2, 0.3]
datasets = [ train_test_split(data_X, data_y.values, test_size=test_size)
             for test_size in test_sizes ]

In [9]:
rus = RandomUnderSampler()
pca = PCA(n_components=100)
scaler = MinMaxScaler()

for idx, dataset in enumerate(datasets):
    X_train, X_test, y_train, y_test = dataset
    
    X_train, y_train = rus.fit_resample(X_train, y_train)
    print(f'Number of each class: { np.unique(y_train, return_counts=True)[0] }')
    
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    datasets[idx] = (X_train, X_test, y_train, y_test)

Number of each class: [0 1 2]
Number of each class: [0 1 2]


# Models

In [10]:
# make a specified model with desired parameters
def get_model(model_type, params):
    if model_type == 'knn':
        return KNeighborsClassifier(**params)
    elif model_type == 'rf':
        return RandomForestClassifier(criterion='gini', **params)
    elif model_type == 'svm':
        return SVC(kernel='rbf', **params)
    elif model_type == 'mlp':
        return MLPClassifier(**params)
    else:
        return None

# Validation & Results

In [11]:
# display a confusion matrix and the classification report
def show_performance(y_true, y_pred):
    c_matrix = confusion_matrix(y_true, y_pred)
    c_table = pd.DataFrame(c_matrix)
    c_table.columns.name = 'truth\pred'
    display(c_table)
    
    report = classification_report(y_true, y_pred)
    print(report)

In [12]:
def show_cross_validate_report(res):
    report = pd.DataFrame({
        'fit_time': res['fit_time'],
        'score_time': res['score_time'],
        'test_score': res['test_score'],
    })
    display(report)

In [13]:
# train a model with 5-fold cross validation and validate the best model with the testing set
def train_model(model_type, param_grid, datasets):
    for test_size, dataset in zip(test_sizes, datasets):
        display(Markdown(f'### Test size: { test_size }'))
        X_train, X_test, y_train, y_test = dataset

        for params in param_grid:
            display(Markdown(f'#### { params }'))
            model = get_model(model_type, params)
            res = cross_validate(model, X_train, y_train, return_estimator=True)

            display(Markdown('#### Training Performance:'))
            show_cross_validate_report(res)

            best_model = res['estimator'][res['test_score'].argmax()]
            y_pred = best_model.predict(X_test)

            display(Markdown('#### Testing Performance:'))
            show_performance(y_test, y_pred)

## KNN

In [14]:
param_grid = ParameterGrid({
    'n_neighbors': [5, 10, 15]
})
train_model('knn', param_grid, datasets)

### Test size: 0.2

#### {'n_neighbors': 5}

#### Training Performance:

,fit_time,score_time,test_score
0,0.591357,27.756326,0.712545
1,0.589011,24.303007,0.724838
2,0.591559,24.698927,0.713862
3,0.600424,18.960065,0.726704
4,0.585859,25.054924,0.726375


#### Testing Performance:

...

#### Training Performance:

,fit_time,score_time,test_score
0,0.524993,24.687350,0.725561
1,0.524052,25.362343,0.723441
2,0.511130,20.247196,0.724813
3,0.522579,23.656578,0.715800
4,0.514059,26.158248,0.715052


#### Testing Performance:

truth\pred,0,1,2
0,13024,1580,3216
1,919,4214,462
2,1634,532,4419


              precision    recall  f1-score   support

           0       0.84      0.73      0.78     17820
           1       0.67      0.75      0.71      5595
           2       0.55      0.67      0.60      6585

    accuracy                           0.72     30000
   macro avg       0.68      0.72      0.70     30000
weighted avg       0.74      0.72      0.73     30000



## Random Forest

In [15]:
param_grid = ParameterGrid({
    'min_samples_leaf': [1, 5, 10]
})
train_model('rf', param_grid, datasets)

### Test size: 0.2

#### {'min_samples_leaf': 1}

#### Training Performance:

,fit_time,score_time,test_score
0,45.675269,0.227156,0.886730
1,46.613976,0.223292,0.894852
2,45.547970,0.224702,0.885303
3,46.398122,0.226215,0.888816
4,45.338431,0.227250,0.886401


#### Testing Performance:

...

#### Training Performance:

,fit_time,score_time,test_score
0,34.234166,0.160088,0.876060
1,33.971726,0.159327,0.875935
2,34.160286,0.158448,0.873691
3,34.014092,0.170868,0.882030
4,33.202309,0.159791,0.876543


#### Testing Performance:

truth\pred,0,1,2
0,15214,499,2107
1,507,4965,123
2,713,8,5864


              precision    recall  f1-score   support

           0       0.93      0.85      0.89     17820
           1       0.91      0.89      0.90      5595
           2       0.72      0.89      0.80      6585

    accuracy                           0.87     30000
   macro avg       0.85      0.88      0.86     30000
weighted avg       0.88      0.87      0.87     30000



## SVM

In [16]:
param_grid = ParameterGrid({
    'C': [1, 5, 10]
})
train_model('svm', param_grid, datasets)

### Test size: 0.2

#### {'C': 1}

#### Training Performance:

,fit_time,score_time,test_score
0,164.038457,29.163267,0.876194
1,164.236164,28.993810,0.884535
2,167.390821,28.735543,0.877072
3,163.191633,28.485504,0.875206
4,166.370654,28.363239,0.883108


#### Testing Performance:

...

#### Training Performance:

,fit_time,score_time,test_score
0,84.538599,14.932498,0.911721
1,84.879532,14.617152,0.912344
2,84.498279,14.545146,0.911970
3,83.542183,14.406843,0.913705
4,85.667024,14.404963,0.911086


#### Testing Performance:

truth\pred,0,1,2
0,15523,574,1723
1,634,4929,32
2,125,1,6459


              precision    recall  f1-score   support

           0       0.95      0.87      0.91     17820
           1       0.90      0.88      0.89      5595
           2       0.79      0.98      0.87      6585

    accuracy                           0.90     30000
   macro avg       0.88      0.91      0.89     30000
weighted avg       0.91      0.90      0.90     30000



## MLP

In [17]:
param_grid = ParameterGrid({
    'hidden_layer_sizes': [256, 512, 1024]
})
train_model('mlp', param_grid, datasets)

### Test size: 0.2

#### {'hidden_layer_sizes': 256}

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


#### Training Performance:

,fit_time,score_time,test_score
0,291.436260,0.343760,0.908682
1,128.575114,0.302343,0.913621
2,331.683448,0.487561,0.927889
3,344.400716,0.502531,0.915706
4,351.603396,0.411484,0.925804


...

#### Training Performance:

,fit_time,score_time,test_score
0,623.438117,1.044393,0.860599
1,410.221161,0.745225,0.926933
2,535.622451,0.960546,0.901746
3,301.060634,0.531323,0.931288
4,269.034996,0.580155,0.922060


#### Testing Performance:

truth\pred,0,1,2
0,16527,557,736
1,634,4945,16
2,164,1,6420


              precision    recall  f1-score   support

           0       0.95      0.93      0.94     17820
           1       0.90      0.88      0.89      5595
           2       0.90      0.97      0.93      6585

    accuracy                           0.93     30000
   macro avg       0.92      0.93      0.92     30000
weighted avg       0.93      0.93      0.93     30000

